# 第6章：概率图模型

## 6.1 概述

**概率图模型**（probabilistic graphical models）：在概率模型的基础上，使用基于图的方法来表示概率分布（或者概率密度、密度函数），是一种通用化的`不确定性知识`表示和处理方法。

在概率图模型的表达中，结点表示变量，结点之间直接相连的边表示相应变量之间的概率关系。当概率分布$P$被表示成概率图模型之后，可以回答与概率分布$P$有关的问题，如计算条件概率$P(Y| E = e)$：在证据$e$给定的条件下，$Y$出现的边缘概率；推断使$P(X_{1}, X_{2}, \cdots, X_{n} | e)$最大的$(X_{1}, X_{2}, \cdots, X_{n})$的分布，即推断最大后验概率时的分布$\argmax_{X}P(X | e)$。

例如，假设$S$为一个汉语句子，$X$是句子$S$切分出来的词序列，则汉语句子的分词过程可以看成是推断使$P(X | S)$最大的词序列$X$的分布。在词性标注中，可以看作在给定序列$X$的情况下，寻找一组最可能的词性标签分布$T$，使得后验概率$P(T | X)$最大。

根据图模型（graphicalmodels）的边是否有向，概率图模型通常被划分成有向概率图模型和无向概率图模型。

<img src="./img/fig_6_1.jpg" width="500">

**动态贝叶斯网络**（dynamic Bayesian networks，DBN）用于处理随时间变化的动态系统中的推断和预测问题。其中，**隐马尔可夫模型**（hidden Markov model，HMM）在语音识别、汉语自动分词与词性标注和统计机器翻译等若干语音语言处理任务中得到了应用；卡尔曼滤波器在信号处理领域得到了应用。

**马尔可夫网络**（Markov network）又称**马尔可夫随机场**（Markovrandomfield，MRF）。其中，**条件随机场**（conditional random field，CRF）应用于自然语言处理中的序列标注、特征选择、机器翻译等任务；**波尔兹曼机**（Boltzmannmachine）被用于依存句法分析和语义角色标注等。

概率图模型的演变：

* 横向：由点到线（序列结构）、线到面（图结构）。以朴素贝叶斯模型为基础的隐马尔可夫模型用于处理线性序列问题，有向图模型用于解决一般图问题；以逻辑回归模型为基础的线性链式条件随机场用于解决“线式”序列问题，通用条件随机场用于解决一般图问题。

* 纵向：在一定条件下生成式模型（generative mode）转变为判别式模型（discriminative model），朴素贝叶斯模型演变为逻辑回归模型，隐马尔可夫模型演变为以逻辑回归为基础的线性链式条件随机场，生成式有向图模型演变为通用条件随机场。

<img src="./img/fig_6_2.jpg" width="550">

**生成式模型**（产生式模型）与**判别式模型**（区分式模型）的本质区别在于模型中观测序列$x$和状态序列$y$之间的决定关系，前者假设$y$决定$x$，后者假设$x$决定$y$。生成模型以“状态（输出）序列$y$按照一定的规律生成观测（输入）序列$x$”为假设，针对联合分布$p(x, y)$进行建模，并且通过估计使生成概率最大的生成序列来获取y。生成式模型是所有变量的全概率模型，因此可以模拟（生成）所有变量的值。在这类模型中一般都有严格的独立性假设，特征是事先给定的，并且特征之间的关系直接体现在公式中。这类模型的优点是：处理单类问题时比较灵活，模型变量之间的关系比较清楚，模型可以通过增量学习获得，可用于数据不完整的情况。其弱点在于模型的推导和学习比较复杂。

典型生成式模型有：$n$元语法模型、HMM朴素的贝叶斯分类器、概率上下文无关文法等。

**判别式模型**符合传统的模式分类思想，认为$y$由$x$决定，直接对后验概率$p(y | x)$进行建模，从$x$中提取特征、学习模型参数，使得条件概率符合一定形式的最优。这类模型的特征可以任意给定，一般特征是通过函数表示的。这种模型的优点是：处理多类问题或分辨某一类与其他类之间的差异时比较灵活，模型简单，容易建立和学习。其弱点为模型的描述能力有限，变量之间的关系不清楚，而且大多数区分式模型是有监督的学习方法，不能扩展成无监督的学习方法。

典型判别式模型有：最大熵模型、条件随机场、支持向量机、最大熵马尔可夫模型（maximum-entropy Markov model，MEMM）、感知机（perceptron）等。


## 6.2 贝叶斯网络

**贝叶斯网络**[Judea Pearl, 1985]又称为信度网络或信念网络（belief networks），是一种基于概率推理的数学模型，通过概率推理处理不确定性和不完整性问题，其理论基础是贝叶斯公式。

形式上，贝叶斯网络是一个有向无环图（directed acyelic graph，DAG），结点表示随机变量，可以是可观测量、隐含变量、未知参量或假设等；结点之间的有向边表示条件依存关系，箭头指向的结点依存于箭头发出的结点（父结点）。*两个结点没有连接关系表示两个随机变量能够在某些特定情况下条件独立，而两个结点有连接关系表示两个随机变量在任何条件下都不存在条件独立。*条件独立是贝叶斯网络所依赖的一个核心概念。每一个结点都与一个概率函数相关，概率函数的输入是该结点的父结点所表示的随机变量的一组特定值，输出为当前结点表示的随机变量的概率值。*概率函数值的大小表达的是结点之间依存关系的强度。*假设父结点有$n$个布尔变量，概率函数可表示成由$2^{n}$个条目构成的二维表，每个条目是其父结点各变量可能的取值(“T"或“F”)与当前
结点真值的组合。例如，如果一篇文章是关于南海岛屿的新闻(将这一事件记作
“News"),文章可能包含介绍南海岛屿历史的内容(这一事件记作“History"),但一般不
会有太多介绍旅游风光的内容(将事件“有介绍旅游风光的内容”记作“Sightseeing")。我
们可以构造-个简单的贝叶斯网络,如图6-3所示。
在这个例子中,“文章是关于南海岛屿的新闻”这一-事件直接影响“有介绍旅游风光的
内容”这一事件。如果分别用N、H.S表示这三个事件,每个变量都有两种可能的取值
